# Homework 2 Part 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [8]:
train = pd.read_csv('/Users/gracegupta/Desktop/grace_gupta_HW1/train.csv')

def preprocessing_pipeline(train):
    ## Your code goes here
    
    #get rid of NAs
    train = train.dropna()
    
    #get rid of outliers
    for X in train:
        mu = np.mean(train[X])
        std = np.std(train[X])
        normalized_data = (train[X]-mu)/std
        indexes = normalized_data < 3
        processed_train = train.loc[indexes, :]
    
    return processed_train

In [9]:
processed_train = preprocessing_pipeline(train)

In [10]:
processed_train.describe() #check that it's the same as part(a)

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000
mean,558.989130,8.327446,0.531943,0.270444,2.538451,0.087760,15.889946,46.425725,0.996784,3.312147,0.656821,10.416244,5.633152
std,323.789428,1.749430,0.180609,0.194891,1.286773,0.046592,10.358023,33.139474,0.001891,0.155899,0.166352,1.068044,0.805931
min,0.000000,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,278.750000,7.100000,0.400000,0.097500,1.900000,0.070000,7.000000,22.000000,0.995697,3.210000,0.550000,9.500000,5.000000
50%,557.500000,7.900000,0.520000,0.260000,2.200000,0.080000,14.000000,38.000000,0.996790,3.310000,0.620000,10.100000,6.000000
75%,840.250000,9.200000,0.640000,0.430000,2.600000,0.091000,22.000000,61.000000,0.997885,3.402500,0.730000,11.100000,6.000000
max,1118.000000,15.900000,1.580000,1.000000,15.400000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [11]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1-r_squared)
    return vif

c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [52]:
import statsmodels.api as sm

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    data = []
    for X in processed_train:
        X_list = []
        X_list.append(X)
        i = processed_train.columns.get_loc(X)
        reg_list = [] #contains all other variables for regression against X
        for j in range(len(processed_train.columns)):
            if j != i :
                reg_list.append(processed_train.columns[j])
        model = sm.OLS(processed_train[X], sm.add_constant(processed_train[reg_list])).fit()
        rsq = model.rsquared
        vif = calculate_vif(rsq)
        X_list.append(vif)
        data.append(X_list)
        
    vif_dataframe = pd.DataFrame(data, columns = ['Variable name','VIF'])
                                
    return vif_dataframe

In [53]:
vif_dataframe = generate_vif_dataframe(processed_train)
vif_dataframe

,Variable name,VIF
0,Id,1.015881
1,fixed acidity,7.724307
2,volatile acidity,1.893183
3,citric acid,3.163935
4,residual sugar,1.707016
5,chlorides,1.537561
6,free sulfur dioxide,1.976494
7,total sulfur dioxide,2.204303
8,density,6.159842
9,pH,3.398952


d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [8]:
## Your code goes here


In [ ]:
## Your discussion goes here

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [ ]:
## Your code goes here

b. Fit a multiple linear regression model and print a summary of the regression result.

In [ ]:
## Your code goes here

c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [ ]:
## Your code goes here

d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [ ]:
## Your code goes here

e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [ ]:
## Your discussion here

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [ ]:
## Your discussion here